# **DD-finite functions in Sage**

In this notebook we provide a brief tutorial explanation (and interactive) to use appropriately the Sage package *dd_functions*. We will cover the following parts:
1. How to install the package (good and bad practices)
2. Basics of the package
3. Operations with the package
    * Arithmetic operations
    * Differential operations
    * Sequence operations
    * Composition
4. Checking identities
5. Built-in examples

## **1. Installing the package**
There are three different ways to obtain the package.

#### **Public _git_ repository**
The package can be obtained freely from the public _git_ repository on GitHub: click [here](https://github.com/Antonio-JP/dd_functions) for the webpage view or clone the repository by [https](https://github.com/Antonio-JP/dd_functions.git) or download the last [zip](https://github.com/Antonio-JP/dd_functions/archive/master.zip).

* This method allow the user to get the <font color="green">very last version</font> of the code.
* From time to time, this means the version <font color="red">is not stable</font>.

#### **Zip download from Webpage**
The last <font color="green">stable version</font> of the code is always available on my [personal webpage](https://www.dk-compmath.jku.at/people/antonio). Just download it and unpack it. To update the code, <font color="red">redownload the zip file and update the folder manually</font>.

##### **How to actually use the package**
Once the repository is cloned or the zip is unpacked, one can run Sage inside the folder or add it to the standars PATH for Sage for look for packages modifying the file `~/.sage/sage.init` and adding the following lines:

`import sys, os;
sys.path.append(os.path.expanduser("###");`
    
where `###` is the path to the local copy of the package.

#### **PiPy installation**
Another option to install and use the package is the use of _pip_ within _Sage_. To do so you have the following options:
* Use the git repository for installation:
  
  `sage -pip install git+https://github.com/Antonio-JP/dd_functions.git`
* Use the unpacked zip file:
  
  `sage -pip install ###`
  
  where `###` is the path to the unpacked folder.

## **2. Basics of the package**
Once the package is completely installed, you can start using it after importing the package into your current Sage session:

In [1]:
%display latex # Remove if you prefer the plain representation
from ajpastor.dd_functions import *

The package is intended to work with an extension for D-finite functions. These functions are power series $f(x)$ that satisfy _linear differential equations_ with **polynomial** coefficients.

Classical examples include the _exponential_ function ($e^x$), _trigonometric_ functions ($\sin(x), \cos(x)$), a large set of _special functions_ (like the Bessel functions, hypergeometric functions, etc.). Several packages are available for working with D-finite functions. The main computable characteristic of this class of power series is that it is enough the coefficients of the differential equation and some initial conditions to completely represent a D-finite function.

In [this paper](https://doi.org/10.1016/j.jsc.2018.07.002), the authors extended the concept of D-finite to the DD-finite functions. A DD-finite function is a power series that satisfies (again) a linear differential equation with **D-finite** coefficients. In this way several non-D-finite examples can be represented with the same structure (a differential equation and some initial conditions): the _double exponential_ ($e^{e^x}$) or the _tangent_ ($\tan(x)$).

However, in [the paper](https://doi.org/10.1016/j.jsc.2018.07.002) it is shown that the only requirement to use this structure of _D.E. + I.C._, the only requirement is a **differential integral domain** $R$. Hence, the set of all power series that satisfy a linear differential equation with coefficients in $R$ is called the set of _differentially definable functions over $R$_. The package provides the implementation to work with these objects and perform different type of operations with them.

##### **Creating differentially definable rings**

This package follows the Parent-Element paradigm followed in Sage, and the user can create any _differentially definable ring_ providing the original ring $R$ for the coefficients of the differential equations and the derivative that works on $R$. The class `DDRing` do the job:

In [2]:
F.<s2> = NumberField(x^2-2); # Coefficients will be rational numbers with sqrt(2)
R.<x> = F[]; # Creating the polynomial ring with those coefficients
A = DDRing(R) # Creating the Diff. definable ring

In [3]:
A # Printing the new ring

DD-Ring over (Univariate Polynomial Ring in x over Number Field in s2 with defining polynomial x^2 - 2)

It is interesting to remark that the package provides by default the differentially definable rings over the polynomial ring ($\mathbb{Q}[x]$) in the variable `DFinite` and the D-finite ring ($\text{D}(\mathbb{Q}[x])$) in the variable `DDFinite`.

In [5]:
print DFinite == DDRing(QQ[x]) ## Checking DFinite are the D-finite functions
DFinite

True


DD-Ring over (Univariate Polynomial Ring in x over Rational Field)

In [6]:
print DDFinite == DDRing(DFinite) ## Checking DFinite are the D-finite functions
DDFinite

True


DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

##### **Creating differentially definable functions**
Once you have a differentially definable ring, you can create any of its elements using the method `element` of the class DDRing. This method requires two arguments:
* A list, representing the coefficients of the linear differential equation.
* A list, with the initial values that defines the function.

For example, we can create the exponential function from the `DFinite` object:

In [7]:
exponential = DFinite.element([-1,1], [1])
exponential

(1:1:1)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

We can also create the sine function and the cosine function:

In [11]:
sine = DFinite.element([1,0,1],[0,1]); show(sine)
cosine = DFinite.element([1,0,1],[1,0]); show(cosine)

(2:2:2)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

(2:2:2)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

Or we can create also DD-finite elements (like the _tangent_):

In [12]:
tangent = DDFinite.element([-2, 0, cosine^2], [0,1]);
tangent

(2:2:7)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

And we can compute with this new elements without any further problem

In [13]:
sine^2

(3:3:3)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

In [14]:
sine - cosine*exponential

(4:4:4)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

## **3. Operations in the package**
With these differentially definable functions we can perform a lot of operations using only the structure of _D.E + I.V_. We can classify the operations into the following categories:

* **Arithmetic operations**: addition (`+`, `-`), multiplication (`*`), powers (`^`)  

In [15]:
# Addition with a polynomial (automatic coercion to DFinite ring)
exponential - 1 - 1/2*x^2

(3:3:5)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Number Field in s2 with defining polynomial x^2 - 2))

In [16]:
# Multiplication between DD-finite functions
tangent * cosine

(4:4:35)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

In [17]:
# A power of an element
tangent^2

(3:3:12)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

* **Differential operations**: with the methods `derivative` and `integrate`

In [18]:
# Fundamental theorem of calculus:
tangent.integrate().derivative() == tangent

True

* **Sequence operations**: we can extract the initial values of the function (methods `getInitialValue` and `getInitialValueList`) and the sequence of the power series (methods `getSequenceElement` and `getSequenceList`). On other words, if we have a function `f` and we consider the sequence $(a_n)_{n \geq 0}$ where $a_n$ is obtained by `f.getSequenceElement(n)`, then `f` is the _ordinary generating function_ of $(a_n)$. On the other hand, if $(b_n)_{n\geq 0}$ is the sequence where $b_n$ is obtained by `f.getInitialValue(n)`, then `f` is the _exponential generating function_ of $(b_n)$.

In [19]:
exponential.getInitialValueList(10)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [20]:
exponential.getSequenceList(10)

[1, 1, 1/2, 1/6, 1/24, 1/120, 1/720, 1/5040, 1/40320, 1/362880]

* **Composition**: whenever a function has $f(0) = 0$, we can compute the composition as power series. But usually the result is not in the same differentially definable ring, but in a more complex one. For example, the composition of two D-finite functions is DD-finite (see [this paper](https://www.dk-compmath.jku.at/publications/dk-reports/2019-02-26/view) for further information). The package take care of deciding where the final element will belong, but from the user perspective, the composition can be performed directly using the _magic call_ method of Python (i.e., using the object as a method):

In [21]:
exponential(exponential - 1)

(1:1:3)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

In [22]:
cosine(sine)

(2:2:10)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

## **4. Checking identities**
One of the main use for this package is to check identities for objects represented with differential equations. For doing so, we can use the equality in Python (`==`) to check if two objects are the same.

For example, we saw previously what is the product of the tangent and the cosine:

In [23]:
tangent * cosine

(4:4:35)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

Which it seems quite a mess. However, the initial conditions look similar to the sine function (in fact, we know they are the same). We can check that directly using the `==` symbol:

In [24]:
tangent * cosine == sine

True

We can check other type of identities:

In [25]:
# sin(sin(x))^2 + cos(sin(x))^2 == 1
sine(sine)^2 + cosine(sine)^2 == 1

True

In [26]:
# tan(x)' == tan(x)^2 + 1
tangent.derivative() == tangent^2 + 1

True

We can also prove other type of identities. For example, if we look to the [Mathieu differential equation](http://mathworld.wolfram.com/MathieuFunction.html):
$$w''(x) + (a - 2q\cos(2x))w(x) = 0,$$
and consider the two fundamental solutions $w_1(x)$ and $w_2(x)$ where $w_1(0) = 1, w_1'(0) = 0$ and $w_2(0) = 0, w_2'(0) = 1$, then we know that the _Wronskian_ must be 1. We can do that quickly with this package:

In [27]:
# Building the functions w_1 and w_2. We use the method MathieuD (see below)
w1 = MathieuD('a','q',(1,0));
w2 = MathieuD('a','q',(0,1));

W = w1*w2.derivative() - w2*w1.derivative();
W

1

It really seems to be equal to 1. We can check that using the equality method in Python

In [28]:
W == 1

True

## **5. Built-in functions in the package**
The package provides several built-in examples of D-finite and DD-finite functions that can be built easily fixing a couple of parameters depending on the function.

Once we have imported the package, a module `ddExamples` is available to get all the information about the built-in functions:

In [31]:
ddExamples?

For each of these functions we can call the documentation using the `?` syntax from Python. Each of them explains briefly which function they are representing, some links to more proper definitions and the paramters required to initialize the object.

We can check that all the functions we have defined through the notebook are the same as the built-in functions:

In [35]:
print "Exponential: ", Exp(x) == exponential;
print "Sine: ", Sin(x) == sine;
print "Cosine: ", Cos(x) == cosine;
print "Tangent: ", Tan(x) == tangent;
print "Double exponential: ", Exp(Exp(x)-1) == exponential(exponential-1);


Exponential:  True
Sine:  True
Cosine:  True
Tangent:  True
Double exponential:  True


## **Time to experiment**
Now feel free to experiment with the package. Any bug, feature request or suggestion is welcome in the email: ajpastor@risc.uni-linz.ac.at